In [1]:
import numpy as np
import os

from app.environment.dataprovider import DataProvider
from app.preparation.preparator import DataPreparator

In [63]:
apikey = os.getenv('TIINGO_API_KEY')
days = 5
weeks = 5
ticker = 'MSFT'

train_start_date = '2000-01-01'
train_end_date = '2010-12-31'

val_start_date = '2011-01-01'
val_end_date = '2015-12-31'

test_start_date = '2016-01-01'
test_end_date = '2020-12-31'

provider = DataProvider(apikey)

In [64]:
def calculate_weekday_mean_change(window):
    day = window['weekday'].values[-1]
    mean_change = window[window['weekday'] == day]['change'].mean()
    return mean_change

def calculate_weekday_sum_change(window):
    day = window['weekday'].values[-1]
    sum_change = window[window['weekday'] == day]['change'].sum()
    return sum_change

def calculate_weekday_sum_ratio(window):
    day = window['weekday'].values[-1]
    sum_change = window[window['weekday'] == day]['change'].sum()
    total_change = window['change'].sum()
    return sum_change / total_change

def calculate_weekday_stdev_change(window):
    day = window['weekday'].values[-1]
    stdev_change = window[window['weekday'] == day]['change'].std()
    return stdev_change


In [147]:
train_quotes = provider.load(ticker, train_start_date, test_end_date)
train_quotes['window'] = train_windows = \
    DataPreparator.calculate_windows(
        train_quotes,
        days=days,
        normalize=True,
        columns=['open', 'high', 'low', 'close'],
        adjust=provider.adjust_prices)
windows = np.array(train_quotes['window'].values.tolist())
train_quotes['change'] = train_quotes['adj_close'].pct_change(1).shift(-1).fillna(0.0)
train_quotes['weekday'] = train_quotes['date'].dt.weekday
train_quotes['weekday_mean_change'] = [calculate_weekday_mean_change(win) for win in train_quotes.rolling(weeks * 5)]
train_quotes['weekday_sum_change'] = [calculate_weekday_sum_change(win) for win in train_quotes.rolling(weeks * 5)]
train_quotes['weekday_sum_ratio'] = [calculate_weekday_sum_ratio(win) for win in train_quotes.rolling(weeks * 5)]
train_quotes['weekday_stdev_change'] = [calculate_weekday_stdev_change(win) for win in train_quotes.rolling(weeks * 5)]
shape = windows.shape
shape

(5284, 1, 4, 5)

In [148]:
# train_quotes[['date', 'weekday', 'adj_close', 'change', 'weekday_mean_change', 'weekday_sum_change',
#              'weekday_sum_ratio', 'weekday_stdev_change']]

In [149]:
# data = train_quotes[['date', 'adj_close', 'weekday_sum_ratio']][-200:].copy()
# data.set_index('date', inplace=True)
# ax = data['adj_close'].plot(figsize=(20, 10))
# data['weekday_sum_ratio'].plot(secondary_y=True, ax=ax)

In [150]:
data = train_quotes[['date', 'adj_close', 'weekday_sum_ratio']][weeks * 5 - 1:].copy()
data['next_close'] = data['adj_close'].shift(-1)
data['last_weekday_sum_ratio'] = data['weekday_sum_ratio'].shift(1)
data['buy'] = data.apply(lambda r: r['adj_close'] if 0.5 < r['weekday_sum_ratio'] < 5 < r['last_weekday_sum_ratio'] else np.nan, axis=1)
data['sell'] = data.apply(lambda r: r['adj_close'] if r['last_weekday_sum_ratio'] < r['weekday_sum_ratio'] < 0.0 else np.nan, axis=1).fillna(method='bfill')
data['ratio'] = data['next_close'] / data['buy']
total_earnings = 1.0
positive_trades = 0
all_trades = 0
for index, row in data[data['ratio'] > 0.0][['ratio', 'weekday_sum_ratio', 'last_weekday_sum_ratio']].iterrows():
    earnings = row['ratio'] - 1.0
    print(f'{earnings:.4f} {row["weekday_sum_ratio"]} {row["last_weekday_sum_ratio"]}')
    positive_trades += 1 if earnings > 0.0 else 0
    all_trades += 1
    total_earnings *= (earnings if earnings < 1.0 else (earnings * (1 - (0.25 * 1.055)))) + 1.0
print(f'{positive_trades} / {all_trades} = {positive_trades / all_trades * 100.0:.2f}% / {(total_earnings - 1.0) * 100.0:.2f}%')

0.0100 4.95731710006193 42.566347778930364
0.0504 0.8243834656347967 7.506918226998267
0.0391 1.4567876020743389 9.718040897382352
-0.0278 0.8150330172099461 9.34094666894476
0.0175 4.45507783424149 39.490725582194784
0.0298 2.748528908704581 8.23873091768309
0.0087 1.8397839919710883 7.505440753115503
0.0061 0.5829826027573498 5.286961926509748
-0.0232 3.392810816065681 5.640843163025953
0.0581 0.9782653497454384 29.28752953909835
-0.0103 3.9614213882493456 7.5330510080872255
0.0084 1.860588424694718 5.415040423913499
0.0053 4.543575798827013 7.913088399243059
0.0147 2.0812832203696248 6.350337846269337
-0.0158 2.440040571465341 33.4807231433421
0.0045 1.662577144772936 24.67136195590777
-0.0026 1.5636895273812805 16.979792315810464
0.0060 1.6860429938778252 126.9813291527877
-0.0087 2.8394102019119836 6.203763975635548
-0.0073 1.7778750208082947 41.80735575339817
-0.0197 2.9502939454531316 22.491259437906873
0.0003 2.312299988885258 7.458919588136893
0.0296 1.7082508362232252 85.6162